In [23]:
import pandas as pd
import numpy as np

np.random.seed(1234)

n = 100

data = {
    'age': np.random.randint(18, 65, size=n),  
    'salary': np.random.normal(loc=50000, scale=15000, size=n).astype(int),  
    'department': np.random.choice(['HR', 'Engineering', 'Marketing'], size=n, p=[0.7, 0.2, 0.1])
}

# 데이터프레임 생성
df = pd.DataFrame(data)

df['department'].value_counts()

HR             72
Engineering    19
Marketing       9
Name: department, dtype: int64

In [24]:
from imblearn.over_sampling import RandomOverSampler

# minority = 소수 클래스만 다수 클래스 수준까지 복제 (default)
# 'not minority' = 소수 클래스 이외의 클래스를 모두 복제
# 'not majority' = 다수 클래스 이외의 클래스를 모두 복제
# 'all' = 모든 클래스를 가장 많은 클래스 수준까지 복제
# dict = 원하는 클래스별로 구체적인 수를 지정
# float = 전체 데이터의 소수 클래스 비율을 정수 비율로 맞춤 (ex. 0.5 → minority / majority = 0.5)

oversample = RandomOverSampler(sampling_strategy='minority', random_state=1234)
X_train, y_train = oversample.fit_resample(df[['age', 'salary']], df['department'])

In [25]:
df_resampled = pd.DataFrame(X_train, columns=['age', 'salary'])
df_resampled['department'] = y_train

In [26]:
df_resampled['department'].value_counts()

HR             72
Marketing      72
Engineering    19
Name: department, dtype: int64

In [27]:
# 모든 소수 클래스가 최대 다수 클래스가 되도록 설정
target_counts = {
    'Engineering': df['department'].value_counts()['HR'],  
    'Marketing': df['department'].value_counts()['HR']
}

oversample = RandomOverSampler(sampling_strategy=target_counts, random_state=1234)
X_train, y_train = oversample.fit_resample(df[['age', 'salary']], df['department']) # fit_resample(data, class)

In [28]:
df_resampled = pd.DataFrame(X_train, columns=['age', 'salary'])
df_resampled['department'] = y_train

In [29]:
df_resampled['department'].value_counts()

HR             72
Marketing      72
Engineering    72
Name: department, dtype: int64